In [4]:
def create_permutation_matrix(s):
    s = list(s)
    def key(si):
        return si[0]
    s_p = [(si,i) for i,si in enumerate(s)]
    s_p.sort(key = key)
    l = len(s)
    M = MatrixSpace(ZZ,l)
    P = M([[0]*l]*l)
    for i in range(l):
        P[i,s_p[i][1]] = 1
    return P

In [149]:
def Basis(F0,s = None):
    m,n = F0.dimensions()
    if (s == None):
        s = [0]*m
        
    pi = create_permutation_matrix(s)
    P,L,U = (pi*F0).LU()
    r = U.rank()
    Lr = L[:r,:r]
    G = L[r:,:r]
    K = F0.base_ring()
    pR.<x> = K[]
    Pinv = P.inverse()
    M = block_matrix(2,2,[[x,0],[-G*Lr.inverse(), 1]])*Pinv*pi
    MS = MatrixSpace(ZZ,m)
    u = MS(Pinv)*pi*vector(s) + vector([1]*r+[0]*(m-r))
    return M,u

In [160]:
low_bound = -4
high_bound = 5
m = 4
n = 2

In [161]:
s = [randint(low_bound, high_bound) for i in range(m)]

In [162]:
pR.<x> = GF(7)[]

In [163]:
M = MatrixSpace(pR,m,n)

In [164]:
A = M.random_element()

In [165]:
A.coefficient_matrix(0)

[3 5]
[6 6]
[0 2]
[0 6]

In [173]:
A

[      4*x^2 + 3 4*x^2 + 4*x + 5]
[3*x^2 + 6*x + 6 4*x^2 + 2*x + 6]
[            2*x               2]
[      5*x^2 + x 5*x^2 + 5*x + 6]

In [150]:
MB,u  =Basis(A)

In [195]:
def M_Basis(F,sigma,shift = None):
    pR = F.base_ring()
    x = pR.gen()
    m,n = F.dimensions()
    MS = MatrixSpace(pR,m)
    if (shift == None):
        shift = [0]*m
    P,u = Basis(F.coefficient_matrix(0), shift)
    for k in range(1,sigma):
        F_prime = (x^(-k)*Mat_pol(P)*F).coefficient_matrix(0)
        M,u = Basis(F_prime, u)
        P = M*P
    return P,u

In [196]:
M,u = M_Basis(A,2)

ValueError: inconsistent number of columns: should be 2 but got 4

In [187]:
P = x^(-1)*M*A

In [189]:
type(P)

<class 'sage.matrix.matrix_generic_dense.Matrix_generic_dense'>

In [194]:
parent(A)(P)

[      4*x^2 + 3 4*x^2 + 4*x + 5]
[3*x^2 + 6*x + 6 4*x^2 + 2*x + 6]
[          x + 5         5*x + 4]
[          x + 3         6*x + 3]

In [191]:
P

[      4*x^2 + 3 4*x^2 + 4*x + 5]
[3*x^2 + 6*x + 6 4*x^2 + 2*x + 6]
[          x + 5         5*x + 4]
[          x + 3         6*x + 3]